In [57]:
import pickle
import numpy as np
from scipy import sparse
from os import listdir

# Read in variant coordinates
coord_file = 'bootstrap/1e4_bootstrap/0.pkl'
coordinates = np.asarray([tuple(map(int, x.decode('UTF-8').split('-'))) for x in pickle.load(open(coord_file, 'rb'))], dtype=int)
coordinates = coordinates[coordinates[:, 0] >= 21, :]

order_indices = np.lexsort((coordinates[:, 1], coordinates[:, 0]))
#print(coordinates.shape, coordinates)
#print(coordinates[order_indices, :])

In [55]:
data_dir = '../split_gen_miss'

subcoordinates = []
submatrices = []
for chrom in range(21, 23):

    # pull coordinates for chromosome
    chrom_coords = np.load('%s/chr.%d.gen.coordinates.npy' %(data_dir, chrom))
    to_find = coordinates[order_indices, :]
    to_find = to_find[to_find[:, 0]==chrom, :]
    #print(chrom_coords.shape, to_find.shape)

    # find indices of the variants we're looking for on this chromosome
    pull_indices = np.searchsorted(chrom_coords[:, 1], to_find[:, 1])
    #print(np.hstack((to_find, chrom_coords[pull_indices, :])))

    # load genotype data from .npz
    gen_files = sorted([f for f in listdir(data_dir) if ('chr.%s' % chrom) in f and 'gen.npz' in f])
    whole_chrom = sparse.hstack([sparse.load_npz('%s/%s' % (data_dir, gen_file)) for gen_file in gen_files])
    print(chrom, whole_chrom.shape)
    
    subcoordinates.append(chrom_coords[pull_indices, :])
    submatrices.append(whole_chrom[:, pull_indices])

ordered_full_coords = np.vstack(subcoordinates)
ordered_full_matrix = sparse.hstack(submatrices)
print(ordered_full_matrix.shape)

reverse_order_indices = np.argsort(order_indices)
random_full_coords = ordered_full_coords[reverse_order_indices, :]
random_full_matrix = ordered_full_matrix[:, reverse_order_indices]
#print(ordered_full_matrix.shape)
#print(np.hstack((coordinates, random_full_coords)))

21 (4610, 1153632)
22 (4610, 1096535)
(4610, 741)


NameError: name 'full_matrix' is not defined

In [59]:
sparse.save_npz(coord_file[:-3] + 'iHART', random_full_matrix)